# GeekBrains, Recommendation Systems
# Lesson 1 Homework

**Импорт библиотек**

In [1]:
import numpy as np

## Задание 1
Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания).

### Решение Задания 1

- Kendall’s tau rank correlation coefficient.
- Spearman’s rho rank correlation coefficient.
- Coverage - покрытие датасета рекомендациями, доля.
- Personalization - насколько различны рекомендации для разных пользователей.
- Intra-list similarity - насколько похожи между собой сделанные рекомендации (средняя cosine близость).
- Expected Reciprocal Rank (ERR).
- Pfound (Яндекса).

## Задание 2
Доделать все функции, где стоит комментарий "сделать дома".

### Решение Задания 2

1. **Hit rate@k**

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных).

Дано:

In [2]:
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return int(flags.sum() > 0)

С учетом @k:

In [7]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    return hit_rate(recommended_list[:k], bought_list)

Проверка:

In [4]:
rec_list = [0, 1, 2, 3]
bought_list = [2, 3, 4, 5, 6]

In [5]:
hit_rate_at_k(rec_list, bought_list, k=2)

0

In [8]:
hit_rate_at_k(rec_list, bought_list, k=3)

1

2. **Precision@k**

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)

Дано:

In [43]:
def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

С учетом @k:

In [9]:
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

Проверка:

In [45]:
precision_at_k(rec_list, bought_list, k=2)

0.0

In [46]:
precision_at_k(rec_list, bought_list, k=3)

0.3333333333333333

In [47]:
precision_at_k(rec_list, bought_list, k=4)

0.5

3. **Recall@k, Money recall@k**

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)

Дано:

In [48]:
def recall(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)

С учетом @k:

In [10]:
def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)

Проверка:

In [50]:
recall_at_k(rec_list, bought_list, k=2)

0.0

In [51]:
recall_at_k(rec_list, bought_list, k=3)

0.2

In [52]:
recall_at_k(rec_list, bought_list, k=4)

0.4

Money recall@k:

In [69]:
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    prices_bought = np.array(prices_bought)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    return np.dot(flags, prices_bought) / prices_bought.sum()

Проверка:

In [65]:
prices_rec = [10, 40, 30, 20]
prices_bought = [30, 20, 5, 60, 15]

In [66]:
money_recall_at_k(rec_list, bought_list, prices_rec, prices_bought, k=2)

0.0

In [67]:
money_recall_at_k(rec_list, bought_list, prices_rec, prices_bought, k=3)

0.23076923076923078

In [68]:
money_recall_at_k(rec_list, bought_list, prices_rec, prices_bought, k=4)

0.38461538461538464

4. **MAP@k**

MAP@k (Mean Average Precision@k) - среднее AP@k по всем юзерам.

Показывает средневзвешенную точность рекомендаций.

$$ MAP@k = \frac{1}{|U|} \sum_u{AP_k} $$
  
|U| - кол-во юзеров

AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка реокмендаций

Напишем AP@k с нуля:

In [98]:
def ap_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    # np.nonzero() - get indexes of nonzero elements of array, returns a tuple with index array as 0th member
    rec_and_relev_idx = np.nonzero(np.isin(recommended_list, bought_list))[0]
    rec_and_relev_count = len(rec_and_relev_idx)
    if rec_and_relev_count == 0:
        return 0
    
    precision_sum = sum([precision_at_k(recommended_list, bought_list, k=idx+1) for idx in rec_and_relev_idx])
    return precision_sum / rec_and_relev_count

Проверка:

In [99]:
ap_k(rec_list, bought_list, k=4)

0.41666666666666663

MAP@k:

In [87]:
def map_k(recommended_list, bought_list, k=5):
    ap_k_sum = sum([ap_k(user_rec_list, user_bought_list, k=k)
                    for user_rec_list, user_bought_list in zip(recommended_list, bought_list)])
    return ap_k_sum / len(recommended_list)

Проверка:

In [100]:
# 3 пользователя
rec_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],   # user 1
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],        # user 2
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]]   # user 3

bought_list_3_users = [[521, 32, 143],            # user 1
                       [143, 156, 991, 43, 11],   # user 2
                       [1, 2]]                    # user 3

In [101]:
map_k(rec_list_3_users, bought_list_3_users, k=1)

0.3333333333333333

In [90]:
map_k(rec_list_3_users, bought_list_3_users, k=3)

0.3333333333333333

In [93]:
map_k(rec_list_3_users, bought_list_3_users, k=10)

0.3333333333333333

5. **NDCG@k**

$$ NDCG = \frac{DCG}{ideal DCG} $$

$$ DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}} $$  

$ discount(i) = i $ if $ i <= 2 $ else $ log_2(i) $   

i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$DCG@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$  
$ideal DCG@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$  

$NDCG = \frac{DCG}{ideal DCG}$

In [102]:
def dcg_discount(item_rank):
    return item_rank if item_rank <= 2 else np.log2(item_rank)

Проверка:

In [104]:
dcg_discount(2)

2

In [105]:
dcg_discount(3)

1.584962500721156

In [144]:
def dcg_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    rec_bought_fact = np.isin(recommended_list, bought_list).astype(int)
    # Ранг начинается с 1
    disc_ranks = np.array([dcg_discount(rank) for rank in range(1, recommended_list.shape[0]+1)])
    
    return (rec_bought_fact / disc_ranks).sum() / recommended_list.shape[0]

Проверка:

In [126]:
dcg_k(rec_list, bought_list, k=2)

0.0

In [127]:
dcg_k(rec_list, bought_list, k=3)

0.2103099178571525

In [128]:
dcg_k(rec_list, bought_list, k=4)

0.2827324383928644

In [133]:
dcg_k(rec_list, rec_list, k=5)

0.6577324383928644

In [137]:
def ndcg_k(recommended_list, bought_list, k=5):
    dcg = dcg_k(recommended_list, bought_list, k=k)
    # Сделав так, получим все '1' в bought_fact
    ideal_dcg = dcg_k(recommended_list, recommended_list, k=k)
    
    return dcg / ideal_dcg

Проверка:

In [138]:
ndcg_k(rec_list, bought_list, k=2)

0.0

In [139]:
ndcg_k(rec_list, bought_list, k=3)

0.2960819109658652

In [142]:
ndcg_k(rec_list, bought_list, k=4)

0.4298593499260986

6. **MRR@k**

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$

$$ RR = \frac{1}{k_u} $$

$$ MRR = mean(\frac{1}{k_u})$$

RR@k:

In [164]:
def reciprocal_rank_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    rec_bought_fact = np.isin(recommended_list, bought_list).astype(int)
    rec_and_relev_idx = np.nonzero(rec_bought_fact)[0]
    rec_and_relev_count = len(rec_and_relev_idx)
    if rec_and_relev_count == 0:
        return 0
    
    # Ранг начинается с 1, а индексы с 0
    return 1 / (rec_and_relev_idx[0] + 1)

Проверка:

In [165]:
reciprocal_rank_k(rec_list, bought_list, k=2)

0

In [166]:
reciprocal_rank_k(rec_list, bought_list, k=3)

0.3333333333333333

In [167]:
reciprocal_rank_k(rec_list, bought_list, k=4)

0.3333333333333333

MRR@k:

In [168]:
def mrr_k(recommended_list, bought_list, k=5):
    rr_k_sum = sum([reciprocal_rank_k(user_rec_list, user_bought_list, k=k)
                   for user_rec_list, user_bought_list in zip(recommended_list, bought_list)])
    return rr_k_sum / len(recommended_list)

Проверка:

In [169]:
mrr_k(rec_list_3_users, bought_list_3_users, k=2)

0.3333333333333333

In [170]:
mrr_k(rec_list_3_users, bought_list_3_users, k=3)

0.3333333333333333

In [171]:
mrr_k(rec_list_3_users, bought_list_3_users, k=4)

0.3333333333333333